In [31]:
import pandas as pd

In [32]:
# 여러 사람의 1년 스케줄을 관리하는 데이터 프레임을 생성
dictionary = {}
dictionary['month'] =['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
dictionary['date'] = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
dictionary['time'] = ['9시에서 12시','1시에서 3시','3시에서 6시']
dictionary['name'] = ['kim', 'lee', 'park', 'choi', 'jung', 'kang', 'yoon', 'lim', 'oh', 'ryu', 'han', 'seo']
dictionary['schedule'] = ['none','meeting','work','vacation']

In [33]:
# name, month, data, time을 키로 하는 
schedule = {}
import random
for i in dictionary['name']:
    for j in dictionary['month']:
        for k in dictionary['date']:
            for l in dictionary['time']:
                # schedule을 랜덤으로 생성
                if i not in schedule:
                    schedule[i] = {}
                if j not in schedule[i]:
                    schedule[i][j] = {}
                if k not in schedule[i][j]:
                    schedule[i][j][k] = {}
                schedule[i][j][k][l] = random.choice(dictionary['schedule'])

In [34]:
import json

In [35]:
schedule_json = json.dumps(schedule, indent=4)

In [36]:
#json 읽기


In [37]:
# json 파일로 저장
with open('schedule.json', 'w') as file:
    file.write(schedule_json)

In [38]:
# schedule.json 읽기
with open('schedule.json', 'r') as file:
    schedule = json.load(file)

In [39]:
# kim lee park 3명이 모두 none인 2월 날자, 시간들을 리스트로 출력
none_list = []
for i in dictionary['date']:
    for j in dictionary['time']:
        if schedule['kim']['mar'][i][j] == 'none' and schedule['lee']['mar'][i][j] == 'none' and schedule['park']['mar'][i][j] == 'none':
            none_list.append((i,j))

In [40]:
none_list

[('7', '1시에서 3시'), ('17', '1시에서 3시')]

In [41]:

import datetime

# 기본적인 날짜와 시간을 담은 dictionary
dictionary = {}
dictionary['month'] =['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
dictionary['date'] = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
dictionary['time'] = ['9시에서 12시','1시에서 3시','3시에서 6시']

# schedule.json 읽어오기
with open('schedule.json', 'r') as file:
    schedule = json.load(file)
    
# 현재 월일 시간을 구하기
now = datetime.datetime.now()

In [42]:
def none_schedule(name:list = ['kim','lee','park'], month:str = 'mar'):
    none_list = []
    num = 0
    for i in dictionary['date']:
        for j in dictionary['time']:
            # name에 있는 사람들이 모두 none인 경우 none_list에 추가
            for k in name:
                if schedule[k][month][i][j] == 'none':
                    num += 1
            if num == len(name):
                none_list.append([i,j])
            num = 0
    return none_list

In [43]:
a = none_schedule(['kim','lee','park'], 'mar')

In [45]:
a

[['7', '1시에서 3시'], ['17', '1시에서 3시']]

# chatbot

In [26]:
generation_config = {
  "temperature": 0.3,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 2048
}

In [27]:
import os
from openai import AzureOpenAI
import time
from dotenv import load_dotenv

dotenv_path = '/home/kic/yskids/service/app/credentials/.env'
load_dotenv(dotenv_path)


class ChatGPTAPI:
    def __init__(self, 
                 generation_config = generation_config, model:str = os.getenv('AZURE_DEPLOYMENT')) -> None:
      self.model = model
      self.generation_config = generation_config
      self.client = AzureOpenAI(api_key = os.getenv('AZURE_API_KEY'),
                                api_version = os.getenv('AZURE_API_VERSION'),
                                azure_endpoint = os.getenv('AZURE_ENDPOINT')
                                )
      
    def gradings(self, guideline:str, self_introduction_1:str, self_introduction_2:str):
      with open('./instructions/grading.txt', 'r') as f:
        data = f.read()
        data = data.format(guideline=guideline, self_introduction_1=self_introduction_1, self_introduction_2=self_introduction_2)
        messages = [{"role": "user", "content": data}]
        start = time.time()
        response = self.client.chat.completions.create(model=self.model, messages=messages, 
                                                  temperature = generation_config['temperature'], 
                                                  top_p = generation_config['top_p'],
                                                  max_tokens = generation_config['max_output_tokens'] )
        end = time.time()
        print(f'generation time: {end-start}')
        output = response.choices[0].message.content
        return output

In [28]:
client = AzureOpenAI(api_key = os.getenv('AZURE_API_KEY'),
                        api_version = os.getenv('AZURE_API_VERSION'),
                        azure_endpoint = os.getenv('AZURE_ENDPOINT')
                        )

In [30]:
# ./instruction.txt 파일의 {information} 부분을 ./company_project_info.txt로 채워넣기
with open('./instruction.txt', 'r') as f:
    data = f.read()
with open('./company_project_info.txt', 'r') as f:
    company_project_info = f.read()
    
data = data.format(information=company_project_info, question = '회사에서 하려는 신규 프로젝트에 대해 구체적으로 알려줘.')
messages = [{"role": "user", "content": data}]
response = client.chat.completions.create(model=os.getenv('AZURE_DEPLOYMENT'), messages=messages, 
                                            temperature = generation_config['temperature'], 
                                            top_p = generation_config['top_p'],
                                            max_tokens = generation_config['max_output_tokens'] )
output = response.choices[0].message.content
print(output)

안녕하세요 지원자님, 회사에 관심 가져주셔서 감사합니다. 저희가 진행하려는 신규 프로젝트는 거대 언어 모델을 활용한 회계 전문용어 번역 모델 개발입니다. 이 프로젝트의 목표는 회계 전문용어의 정확하고 효과적인 번역을 위한 언어 모델을 개발하고, 해당 모델을 기반으로 한 어플리케이션을 통해 다국어 환경에서 회계 문서 및 통역에 필요한 텍스트의 자동 번역을 제공하는 것입니다.

주요 작업 및 책임으로는 거대 언어 모델의 사전 학습을 수행하고, 회계 전문용어에 대한 특화된 학습 데이터셋을 구축하는 것입니다. 또한, 언어 모델을 기반으로 한 어플리케이션을 설계 및 개발하며, 모델의 정확성 및 안정성을 검증하고 최적화하는 작업이 포함됩니다.

이를 위해 사용할 기술로는 최신의 언어 모델 아키텍처를 사용하여 회계 전문 용어에 대한 사전 학습을 수행하고, 강화 학습 및 하이퍼파라미터 최적화(RLHF)를 통해 모델의 정확성을 향상시키며, LangChain 기술을 도입하여 번역 품질을 개선합니다.

예상 결과 및 성과로는 회계 전문용어에 대한 효과적이고 정확한 번역 모델의 개발, 다국어 환경에서의 어플리케이션 안정성과 성능 보장, 그리고 RLHF 및 LangChain 기술을 통한 모델의 우수한 학습 및 번역 능력이 있습니다. 프로젝트 진행 중에는 지속적인 테스트와 개선을 통해 사용자 경험을 향상시키고, 최신 언어 모델 기술에 대한 지속적인 업그레이드가 필요할 것으로 예상됩니다.

더 필요하신 사안 있으시면, 언제든지 질의 부탁드립니다.
